In [17]:
!pip install streamlit ngrok nltk torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [37]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import nltk
from nltk.tree import Tree
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# Download the Treebank corpus
nltk.download("treebank")

# Load the dataset properly
treebank = list(nltk.corpus.treebank.parsed_sents())  # Convert to list

def tree_to_tuples(tree):
    tuples = []
    if isinstance(tree, Tree):
        for subtree in tree:
            tuples.append((tree.label(), subtree.label() if isinstance(subtree, Tree) else subtree))
            tuples.extend(tree_to_tuples(subtree))
    return tuples
class TreeDataset(Dataset):
    def __init__(self, trees):
        self.data = []
        for tree in trees:
            self.data.extend(tree_to_tuples(tree))
        self.labels = list(set([x[0] for x in self.data] + [x[1] for x in self.data if isinstance(x[1], str)]))
        self.label2idx = {label: idx for idx, label in enumerate(self.labels)}
        self.idx2label = {idx: label for label, idx in self.label2idx.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        parent, child = self.data[idx]
        parent_idx = self.label2idx[parent]
        child_idx = self.label2idx.get(child, len(self.label2idx) - 1)  # Ensure index is valid
        return torch.tensor(parent_idx), torch.tensor(child_idx)
class RNNParser(nn.Module):
    def __init__(self, num_labels, embedding_dim=64, hidden_dim=128):
        super(RNNParser, self).__init__()
        self.embedding = nn.Embedding(num_labels + 1, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_labels + 1) # Add +1 here to match the embedding layer output

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        output, _ = self.rnn(x)
        output = self.fc(output.squeeze(1))
        return output

# Assuming 'dataset' is your TreeDataset instance
model = RNNParser(num_labels=len(dataset.labels))

# Assuming you have a DataLoader named 'dataloader' for your dataset
dataloader = DataLoader(dataset, batch_size=16, shuffle=True) # Example batch size

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ... rest of your training and evaluation code ...

# Train model
epochs = 5
for epoch in range(epochs):
    total_loss = 0
    for parent, child in dataloader:
        optimizer.zero_grad()
        output = model(parent)
        loss = criterion(output, child)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

torch.save(model.state_dict(), "parser_model.pth")
# Evaluate model
def evaluate(model, dataset):
    y_true, y_pred = [], []
    for parent, child in dataset:
        with torch.no_grad():
            output = model(parent.unsqueeze(0))
            predicted = torch.argmax(output, dim=1).item()
        y_true.append(child.item())
        y_pred.append(predicted)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

evaluate(model, dataset)

model = RNNParser(num_labels=len(dataset.labels))
model.load_state_dict(torch.load("parser_model.pth", map_location=torch.device("cpu"), weights_only=True))
model.eval()

# Streamlit UI
st.title("Constituency Parser with RNN")
sentence = st.text_input("Enter a sentence:")
if st.button("Parse"):
    if sentence:
        with torch.no_grad():
            input_tensor = dataset.parse_sentence(sentence)
            output = model(input_tensor.unsqueeze(0))
            predicted_idx = torch.argmax(output, dim=1).tolist()
            predicted_labels = [dataset.idx2label[idx] for idx in predicted_idx]
        st.write("Predicted Parse Tree:", predicted_labels)
    else:
        st.warning("Please enter a valid sentence.")



Overwriting app.py


In [38]:
torch.save(model.state_dict(), "parser_model.pth")

In [44]:
!pip install pyngrok
!streamlit run app.py --server.headless true & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.141.115:8501

your url is: https://stupid-ideas-yawn.loca.lt
  Stopping...
^C


In [27]:


# Download the Treebank corpus
nltk.download("treebank")

# Load the dataset properly
treebank = list(nltk.corpus.treebank.parsed_sents())  # Convert to list



[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [28]:
def tree_to_tuples(tree):
    tuples = []
    if isinstance(tree, Tree):
        for subtree in tree:
            tuples.append((tree.label(), subtree.label() if isinstance(subtree, Tree) else subtree))
            tuples.extend(tree_to_tuples(subtree))
    return tuples

In [29]:
class TreeDataset(Dataset):
    def __init__(self, trees):
        self.data = []
        for tree in trees:
            self.data.extend(tree_to_tuples(tree))
        self.labels = list(set([x[0] for x in self.data] + [x[1] for x in self.data if isinstance(x[1], str)]))
        self.label2idx = {label: idx for idx, label in enumerate(self.labels)}
        self.idx2label = {idx: label for label, idx in self.label2idx.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        parent, child = self.data[idx]
        parent_idx = self.label2idx[parent]
        child_idx = self.label2idx.get(child, len(self.label2idx) - 1)  # Ensure index is valid
        return torch.tensor(parent_idx), torch.tensor(child_idx)


In [30]:
class RNNParser(nn.Module):
    def __init__(self, num_labels, embedding_dim=64, hidden_dim=128):
        super(RNNParser, self).__init__()
        self.embedding = nn.Embedding(num_labels + 1, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_labels + 1) # Add +1 here to match the embedding layer output

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        output, _ = self.rnn(x)
        output = self.fc(output.squeeze(1))
        return output

# Assuming 'dataset' is your TreeDataset instance
model = RNNParser(num_labels=len(dataset.labels))

# Assuming you have a DataLoader named 'dataloader' for your dataset
dataloader = DataLoader(dataset, batch_size=16, shuffle=True) # Example batch size

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ... rest of your training and evaluation code ...

# Train model
epochs = 5
for epoch in range(epochs):
    total_loss = 0
    for parent, child in dataloader:
        optimizer.zero_grad()
        output = model(parent)
        loss = criterion(output, child)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

torch.save(model.state_dict(), "parser_model.pth")
# Evaluate model
def evaluate(model, dataset):
    y_true, y_pred = [], []
    for parent, child in dataset:
        with torch.no_grad():
            output = model(parent.unsqueeze(0))
            predicted = torch.argmax(output, dim=1).item()
        y_true.append(child.item())
        y_pred.append(predicted)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

evaluate(model, dataset)

Epoch 1, Loss: 28726.9507
Epoch 2, Loss: 27795.2133
Epoch 3, Loss: 27637.3043
Epoch 4, Loss: 27548.0148
Epoch 5, Loss: 27505.8876
Precision: 0.0301, Recall: 0.0216, F1-score: 0.0212


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
model = RNNParser(num_labels=len(dataset.labels))
model.load_state_dict(torch.load("parser_model.pth", map_location=torch.device("cpu")))
model.eval()

# Streamlit UI
st.title("Constituency Parser with RNN")
sentence = st.text_input("Enter a sentence:")
if st.button("Parse"):
    if sentence:
        with torch.no_grad():
            input_tensor = dataset.parse_sentence(sentence)
            output = model(input_tensor.unsqueeze(0))
            predicted_idx = torch.argmax(output, dim=1).tolist()
            predicted_labels = [dataset.idx2label[idx] for idx in predicted_idx]
        st.write("Predicted Parse Tree:", predicted_labels)
    else:
        st.warning("Please enter a valid sentence.")

<ipython-input-31-1e07f62a431b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("parser_model.pth", map_location=torch.device("cpu")))
2025